# GERÇEK BİR VERİ SETİ ÜZERİNDE A/B TESTİ UYGULAMASI

![AB-Testi.png](https://www.vargonen.com/blog/wp-content/uploads/2020/05/ab-testi-nedir.png)

> ****A/B TESTİ,İKİ BAĞIMSIZ UYGULAMANIN BİRBİRİYLE KARŞILAŞTIRILIP,KARŞILAŞTIRILAN PARAMETRELER ARASINDA İSTATİKSEL OLARAK FARK VAR MIDIRIN CEVABINI VERİR.**

**İŞ PROBLEMİMİZ:
Bir e-ticaret şirketi bir platformda satış yapıyor, satış yaptığı  sitenin giriş ekranında değişikliğe gitmek istiyor ve satışlarda, ürün görüntülemelerinde ve tıklamalarında  istatiksel olarak bir farklılık olup olmadığını bilmek istiyor.**

> **H0: MEVCUT DURUMUN GETİRİSİYLE DEĞİŞİKLİK SONRASI GETİRİ ARASINDA İSTATİKSEL OLARAK ANLAMLI BİR FARK YOKTUR.
> H1:...FARK VARDIR.**

In [21]:

import numpy as np 
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import ttest_1samp, shapiro, levene, ttest_ind, mannwhitneyu
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [22]:
#excel dosyalarını açabilmek için
!pip install openpyxl

In [23]:
#control ve test verilerini okutuyorum
data=pd.read_excel("../input/ab-test/ab_testing.xlsx",sheet_name="Control Group")
data_test=pd.read_excel("../input/ab-test/ab_testing.xlsx",sheet_name="Test Group")

In [24]:
data.head()

,Impression,Click,Purchase,Earning,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,82529.45927,6090.07732,665.21125,2311.27714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,98050.45193,3382.86179,315.08489,1742.80686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,82696.02355,4167.96575,458.08374,1797.82745,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,109914.40040,4910.88224,487.09077,1696.22918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,108457.76263,5987.65581,441.03405,1543.72018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
data=data[['Impression', 'Click', 'Purchase', 'Earning']]

In [26]:
data.head()

,Impression,Click,Purchase,Earning
0,82529.45927,6090.07732,665.21125,2311.27714
1,98050.45193,3382.86179,315.08489,1742.80686
2,82696.02355,4167.96575,458.08374,1797.82745
3,109914.40040,4910.88224,487.09077,1696.22918
4,108457.76263,5987.65581,441.03405,1543.72018


In [27]:
data_test=data_test[['Impression', 'Click', 'Purchase', 'Earning']]

In [28]:
#satın alım sayılarının her iki veride de ortalamasına bakıyorum.
data["Purchase"].mean() 

550.8940587702316

In [29]:
data_test["Purchase"].mean()

582.1060966484677

İki veri arasında matematiksel olarak fark var bir de istatiksel olarak var mı ona bakalım.

In [30]:
#normallik testini kontrol verisindeki purchase değerlerine uyguluyorum.
test_stat, pvalue = shapiro(data['Purchase'])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.9773, p-value = 0.5891


In [31]:
#normallik testini test veri setindeki purchase değerlerine uyguluyorum.
test_stat, pvalue = shapiro(data_test['Purchase'])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.9589, p-value = 0.1541


In [32]:
#varyansların homojen olup olmadığını kontrol ediyorum
test_stat, pvalue = levene(data["Purchase"],data_test["Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 2.6393, p-value = 0.1083


p-value değerleri 0.05'ten büyük olduğundan h0 reddedilemez ve varsayımlar sağlandığından bağımsız iki örneklem t testi yapılır.

In [33]:
#bağımsız iki örneklem t testi uygulaması
#equal_ver=True varyansların homojen olduğunu belirtir varyanslar homojen değilse 
# yani yalnızca normallik varsayımının sağlandığı durumlarda bu parametreye false değeri girilir.
test_stat, pvalue = ttest_ind(data["Purchase"],data_test["Purchase"],
                              equal_var=True)
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = -0.9416, p-value = 0.3493


p-value değeri 0.05'ten büyük olduğundan h0 reddedilemez yani mevcut durumun getirisiyle değişiklik sonrası getiri arasında istatiksel olarak anlamlı bir fark yoktur.

In [34]:
#a/b testini fonksiyonlaştırıyorum.
def statistics(dataframe1,dataframe2,transaction1):
    p=0.05
    test_stat, pvalue1 = shapiro(dataframe1[transaction1])
    test_stat, pvalue2 = shapiro(dataframe2[transaction1])
    test_stat, pvalue3 = levene(dataframe1[transaction1], dataframe2[transaction1])
    if pvalue1>p and pvalue2>p and pvalue3>p:
        test_stat, pvalue4 = ttest_ind(dataframe1[transaction1], dataframe2[transaction1],equal_var=True)
        print(f"pvalue değeri:{pvalue4}")
        if pvalue4 > p:
            print(f"{pvalue4} bu sayı {p}'den büyüktür, H0 reddedilemez")
        else:
            print(f"{pvalue4} bu sayı {p}'den küçüktür, H0 red")
    elif pvalue1>p and pvalue2>p and pvalue3<p:
        test_stat, pvalue4 = ttest_ind(dataframe1[transaction1], dataframe2[transaction1], equal_var=False)
        print(f"pvalue değeri:{pvalue4}")
        if pvalue4 > p:
            print(f"{pvalue4} bu sayı {p}'den büyüktür, H0 reddedilemez")
        else:
            print(f"{pvalue4} bu sayı {p}'den küçüktür,H0 red")
    elif (pvalue1>p and pvalue2<p) or (pvalue1<p and pvalue2>p) or (pvalue1<p and pvalue2<p):
        test_stat, pvalue = mannwhitneyu(dataframe1[transaction1],
                                         dataframe2[transaction1])
        print(f"pvalue değeri:{pvalue}")
        if pvalue>p:
            print(f"{pvalue} bu sayı {p}'den büyüktür, H0 reddedilemez")
        else:
            print(f"{pvalue} bu sayı {p}'den küçüktür, H0 red")

In [35]:
#güzel oldu :)
statistics(data,data_test,"Purchase")

pvalue değeri:0.34932579202108416
0.34932579202108416 bu sayı 0.05'den büyüktür, H0 reddedilemez


In [36]:
def statistics2(dataframe1,dataframe2,transaction1,transaction2):
    p=0.05
    test_stat, pvalue1 = shapiro(dataframe1.groupby(transaction1)[transaction2].mean())
    test_stat, pvalue2 = shapiro(dataframe2.groupby(transaction1)[transaction2].mean())
    test_stat, pvalue3 = levene(dataframe1.groupby(transaction1)[transaction2].mean(),
                                dataframe2.groupby(transaction1)[transaction2].mean())
    if pvalue1>p and pvalue2>p and pvalue3>p:
        test_stat, pvalue4 = ttest_ind(dataframe1.groupby(transaction1)[transaction2].mean(),dataframe2.groupby(transaction1)[transaction2].mean(),equal_var=True)
        print(f"pvalue değeri:{pvalue4}")
        if pvalue4 > p:
            print(f"{pvalue4} bu sayı {p}'den büyüktür, H0 reddedilemez")
        else:
            print(f"{pvalue4} bu sayı {p}'den küçüktür, H0 red")
    elif pvalue1 > p and pvalue2 > p and pvalue3 < p:
        test_stat, pvalue4 = ttest_ind(dataframe1.groupby(transaction1)[transaction2].mean(),dataframe2.groupby(transaction1)[transaction2].mean(), equal_var=False)
        if pvalue4 > p:
            print(f"{pvalue4} bu sayı {p}'den büyüktür, H0 reddedilemez")
        else:
            print(f"{pvalue4} bu sayı {p}'den küçüktür, H0 red")
    elif (pvalue1>p and pvalue2<p) or (pvalue1<p and pvalue2>p) or (pvalue1<p and pvalue2<p):
        test_stat, pvalue4 = mannwhitneyu(dataframe1.groupby(transaction1)[transaction2].mean(),
                                          dataframe2.groupby(transaction1)[transaction2].mean())
        if pvalue4 > p:
            print(f"{pvalue4} bu sayı {p}'den büyüktür, H0 reddedilemez")
        else:
            print(f"{pvalue4} bu sayı {p}'den küçüktür, H0 red")


> *  **H0:  Mevcut durumda tıklanmalara karşılık satın alım sayısının ortalaması ile değişiklik sorası tıklanmalara karşılık satın alım sayısının ortalaması arasında istatiksel olarak anlamlı bir fark yoktur.****
> * **H1:  ...fark vardır.****

In [37]:
data.groupby("Click")[["Purchase"]].mean().sum()

Purchase   22035.76235
dtype: float64

In [38]:
data_test.groupby("Click")[["Purchase"]].mean().sum()

Purchase   23284.24387
dtype: float64

ikisi arasında matematiksel olarak farklılık var.

In [39]:
statistics2(data,data_test,"Click","Purchase")

pvalue değeri:0.34932579202108416
0.34932579202108416 bu sayı 0.05'den büyüktür, H0 reddedilemez


> **HER İKİ ANALİZDE DE H0 REDDEDİLEMEDİĞİNDEN ARALARINDA İSTATİKSEL OLARAK ANLAMLI BİR FARKLILIK YOKTUR DİYOR VE MEVCUT(ESKİ) YÖNTEMLE DEVAM EDİYORUZ.**